### Image Classification with the MNIST Dataset

#### The MNIST Dataset.

+ The MNIST dataset consists of 70,000 grayscale images of handwritten digits (0-9), split into 60,000 training images, and 10,000 test images. Each image is 28x28 pixels (784 pixels total) with a digit from 0 to 9
+ Common uses: training and testing ML models; benchmarking new algorithms; computer vision

In [19]:
import os
if not os.path.exists('./data'):
    os.makedirs('./data')
import torch
from torchvision import datasets, transforms
!uv pip install matplotlib

# define transformations
## creates a pipeline of transformations that will be applied to each image
    ## converts images to PyTorch tensors
    ## standardizes the data using MNIST's mean (0.1307) and standard deviation (0.3081)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
train_dataset = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

Using Python 3.12.7 environment at: /Users/brendan/Desktop - Brendan’s MacBook Air/nvda-certification/torch-ml-312
Audited 1 package in 4ms


### Create a simple Neural Network

In [20]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Input images are 28x28 pixels = 784 input features
        self.fc1 = nn.Linear(784, 128)  # First layer: 784 -> 128
        self.fc2 = nn.Linear(128, 10)   # Output layer: 128 -> 10 (digits 0-9)
    
    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the image
        x = F.relu(self.fc1(x))  # Apply first layer and ReLU activation
        x = self.fc2(x)  # Apply final layer
        return F.log_softmax(x, dim=1)

# Create the model
model = SimpleNet()

#### Then an Optimizer and loss function

In [21]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.01)  # Simple gradient descent
criterion = nn.CrossEntropyLoss()  # Standard loss for classification

#### Simple Training Loop

In [22]:
def train_one_epoch(model, train_loader, optimizer, criterion):
    # Set model to training mode
    model.train()
    
    # Loop through each batch from our data loader
    for batch_idx, (data, target) in enumerate(train_loader):
        # Clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        # Forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # Calculate loss
        loss = criterion(output, target)
        
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # Update the parameters
        optimizer.step()
        
        # Print progress every 100 batches
        if batch_idx % 50 == 0:
            print(f'Training: Batch {batch_idx}/{len(train_loader)} Loss: {loss.item():.4f}')

# Set number of epochs (how many times we'll train on the entire dataset)
n_epochs = 2

# Training loop
for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/{n_epochs}')
    train_one_epoch(model, train_loader, optimizer, criterion)
    print('-------------------')

Epoch 1/2
Training: Batch 0/938 Loss: 2.2735
Training: Batch 50/938 Loss: 1.5817
Training: Batch 100/938 Loss: 0.9992
Training: Batch 150/938 Loss: 0.7056
Training: Batch 200/938 Loss: 0.6283
Training: Batch 250/938 Loss: 0.4541
Training: Batch 300/938 Loss: 0.4949
Training: Batch 350/938 Loss: 0.5259
Training: Batch 400/938 Loss: 0.5456
Training: Batch 450/938 Loss: 0.5614
Training: Batch 500/938 Loss: 0.3118
Training: Batch 550/938 Loss: 0.5109
Training: Batch 600/938 Loss: 0.5193
Training: Batch 650/938 Loss: 0.4075
Training: Batch 700/938 Loss: 0.2762
Training: Batch 750/938 Loss: 0.1576
Training: Batch 800/938 Loss: 0.3511
Training: Batch 850/938 Loss: 0.2069
Training: Batch 900/938 Loss: 0.4180
-------------------
Epoch 2/2
Training: Batch 0/938 Loss: 0.2968
Training: Batch 50/938 Loss: 0.2617
Training: Batch 100/938 Loss: 0.3633
Training: Batch 150/938 Loss: 0.4551
Training: Batch 200/938 Loss: 0.4526
Training: Batch 250/938 Loss: 0.4451
Training: Batch 300/938 Loss: 0.1470
Trai

### Check Accuracy

In [23]:
def test_accuracy(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    
    # Don't compute gradients during testing
    with torch.no_grad():
        for data, target in test_loader:
            # Get predictions
            outputs = model(data)
            # Get the predicted class (digit)
            _, predicted = torch.max(outputs.data, 1)
            # Count total and correct predictions
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

#### Predict Actual Digits

In [24]:
def predict_digit(model, test_loader):
    model.eval()
    
    # Get a single batch of test data
    dataiter = iter(test_loader)
    images, labels = next(dataiter)
    
    # Get predictions for the first few images
    with torch.no_grad():
        outputs = model(images[:15])  # Let's look at first 15 images
        _, predictions = torch.max(outputs, 1)
    
    # Print results
    for i in range(15):
        print(f'True digit: {labels[i]} | Predicted digit: {predictions[i]}')

# Let's use all of these:
print("Testing model accuracy...")
test_accuracy(model, test_loader)

print("\nSaving model...")
torch.save(model.state_dict(), 'mnist_model.pth')

print("\nMaking some predictions...")
predict_digit(model, test_loader)

Testing model accuracy...
Accuracy: 92.60%

Saving model...

Making some predictions...
True digit: 7 | Predicted digit: 7
True digit: 2 | Predicted digit: 2
True digit: 1 | Predicted digit: 1
True digit: 0 | Predicted digit: 0
True digit: 4 | Predicted digit: 4
True digit: 1 | Predicted digit: 1
True digit: 4 | Predicted digit: 4
True digit: 9 | Predicted digit: 9
True digit: 5 | Predicted digit: 6
True digit: 9 | Predicted digit: 9
True digit: 0 | Predicted digit: 0
True digit: 6 | Predicted digit: 6
True digit: 9 | Predicted digit: 9
True digit: 0 | Predicted digit: 0
True digit: 1 | Predicted digit: 1
